In [131]:
import pandas as pd 
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [132]:
df = pd.read_csv(r"C:\Users\yetes\OneDrive\Desktop\Excel\Titanic-Dataset.csv")

In [133]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,3.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,3.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,3.0


In [134]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True)

In [135]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,3.0
1,1,1,1,38.0,1,0,71.2833,1.0
2,1,3,1,26.0,0,0,7.9250,3.0
3,1,1,1,35.0,1,0,53.1000,3.0
4,0,3,0,35.0,0,0,8.0500,3.0


In [136]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2,random_state=42)

In [137]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,0,45.5,0,0,28.5000,3.0
733,2,0,23.0,0,0,13.0000,3.0
382,3,0,32.0,0,0,7.9250,3.0
704,3,0,26.0,1,0,7.8542,3.0
813,3,1,6.0,4,2,31.2750,3.0


In [138]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,3,0,NaN,1,1,15.2458,1.0
439,2,0,31.0,0,0,10.5000,3.0
840,3,0,20.0,0,0,7.9250,3.0
720,2,1,6.0,0,1,33.0000,3.0
39,3,1,14.0,1,0,11.2417,1.0


In [228]:
y_train.sample(5)

159    0
36     1
225    0
334    1
843    0
Name: Survived, dtype: int64

In [229]:
# Imputation transformer
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), ['Age']),  # Impute missing values for 'Age'
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), ['Embarked'])  # Impute missing values for 'Embarked'
], remainder='passthrough')


In [230]:
# scaling 
trf2 = ColumnTransformer([
    ("scale",MinMaxScaler(),slice(0,10))  # after transformation of above columns, we get the total 10 columns
],remainder='passthrough') 

In [231]:
# feature selection
trf3 = SelectKBest(score_func=chi2, k=7)  # Top 8 IMP column

In [259]:
trf4 = DecisionTreeClassifier()

<h2>Create Pipeline</h2>

In [260]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
])

 <h2> Pipeline Vs make_pipeline</h2>
 <br>
 <p>Pipeline required naming of steps, make_pipeline does not required.</p>
 <br>
     <p>(Same applies to ColumnTransformer vs make_column_transformer</p>

In [261]:
# # Alternate syntax
# pipe = make_pipeline(trf1,trf2,trf3,trf4)

In [262]:
X_train.shape

(712, 7)

In [263]:
# train the model
pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  ['Age']),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Embarked'])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf3',
                 SelectKBest(k=7,
                             score_func=<function chi2 at 0x000001F266712840>)),
                ('trf4', DecisionTreeClassifier())])

In [264]:
# Display Pipeline
from sklearn import set_config
set_config(display='diagram')

<h2>Explore Pipeline</h2>

In [265]:
pipe.named_steps['trf2'].transformers_

[('scale', MinMaxScaler(), slice(0, 10, None))]

In [266]:
y_pred = pipe.predict(X_test)
y_pred

array([0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1], dtype=int64)

In [267]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7877094972067039

<h2>Cross Validation using Pipeline</h2>

In [268]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()   # croos_validation(cv) = 5

0.7569979316458191

<h2>GriSearch using Pipeline</h2>

In [276]:
# gridsearchcv & also hyperparameter
params = {
    'trf4__max_depth': [1, 2, 3,4,5,None]
}


In [277]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         ['Age']),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         ['Embarked'])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf3',
                                        SelectKBest(k=7,
                                                    score_func=<function chi2 at 0x000001F266712840>)),
                                       ('trf4', DecisionTreeClassifier())]),
             param_grid={'trf4__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [279]:
grid.best_score_

0.821609376538954

In [280]:
grid.best_params_

{'trf4__max_depth': 3}

<h2>Exporting the Pipeline for Production</h2>

In [284]:
# Export
import pickle
pickle.dump(pipe,open('1.pipe.pkl','wb'))